In [2]:
import cv2

import numpy as np

cap=cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
skip=0
face_data = [] 
dataset_path="face_dataset/"

file_name=input("Enter the name of person : ")

while True:

    ret,frame=cap.read()

    gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    if ret==False:
        continue

    faces=face_cascade.detectMultiScale(gray_frame, 1.3,5)

    if len(faces)==0:
        continue

    k=1

    faces=sorted(faces, key=lambda x: x[2]*x[3], reverse=True)

    skip=skip+1

    for face in faces[:1]:
        x,y,w,h=face

        offset=5
        face_offset=frame[y-offset:y+h+offset,x-offset:x+w+offset] 
        face_selection=cv2.resize(face_offset, (100, 100))

    if skip%10==0:
        face_data.append(face_selection)
        print (len(face_data))

    cv2.imshow(str(k), face_selection) 
    k+=1

    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0), 2)

    cv2.imshow("faces", frame)
    quit=cv2.waitKey(1)&0xFF
    if quit==ord('q'):
        break
        
face_data=np.array(face_data)
face_data=face_data.reshape(face_data.shape[0],-1)
print(face_data.shape)

np.save(dataset_path+file_name,face_data)


cap.release()
cv2.destroyAllWindows()


Enter the name of person : Arnav
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
(33, 30000)


In [3]:
import cv2
import numpy as np 
import os 

########## KNN CODE ############
def distance(v1, v2):
	# Eucledian 
	return np.sqrt(((v1-v2)**2).sum())

def knn(train, test, k=5):
	dist = []
	
	for i in range(train.shape[0]):
		# Get the vector and label
		ix = train[i, :-1]
		iy = train[i, -1]
		# Compute the distance from test point
		d = distance(test, ix)
		dist.append([d, iy])
	# Sort based on distance and get top k
	dk = sorted(dist, key=lambda x: x[0])[:k]
	# Retrieve only the labels
	labels = np.array(dk)[:, -1]
	
	# Get frequencies of each label
	output = np.unique(labels, return_counts=True)
	# Find max frequency and corresponding label
	index = np.argmax(output[1])
	return output[0][index]
################################

cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


skip = 0
dataset_path = './face_dataset/'

face_data = [] 
labels = []

class_id = 0 
names = {} 


# Data Preparation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id] = fx[:-4]
        print("Loaded "+fx)
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)

        target = class_id*np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data,axis=0)
face_labels = np.concatenate(labels,axis=0).reshape((-1,1))

print(face_dataset.shape)
print(face_labels.shape)

trainset = np.concatenate((face_dataset,face_labels),axis=1)
print(trainset.shape)

while True:
    ret,frame = cap.read()
    if ret == False:
        continue

    faces = face_cascade.detectMultiScale(frame,1.3,5)
    if(len(faces)==0):
        continue

    for face in faces:
        x,y,w,h = face

        offset = 10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))

        #Predicted Label (out)
        out = knn(trainset,face_section.flatten())

        pred_name = names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

    cv2.imshow("Faces",frame)

    key = cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



Loaded Arnav.npy
(33, 30000)
(33, 1)
(33, 30001)
